In [ ]:
#!pip3 install opencv-python

In [1]:
import cv2 #opencv
import os #working with file paths
import time
import uuid

In [ ]:
#!pip3 install tensorflow-object-detection-api

In [2]:
imgPath = 'Tensorflow\workspace\images\collectedimages'

In [3]:
labels = ['hello', 'how', 'yes', 'no', 'nice', 'to', 'meet', 'you', 'me', 
          'thankyou', 'peace', 'love', 'mother', 'father',
          'friend', 'please', 'eatfood', 'sorry', 'write', 'help',
          'goodjob', 'vulcan', 'stand', 'up']
nimg = 20

In [4]:
for label in labels:
    !mkdir {'Tensorflow\workspace\images\collectedimages\\'+label}
    cap = cv2.VideoCapture(0)
    print('Collecting images for {}'.format(label))
    time.sleep (5)
    for imgnum in range(nimg):
        ret, frame = cap.read()
        imgname = os.path.join(imgPath, label, label+','+'{}.jpg'.format(str(uuid.uuid1()))) 
        cv2.imwrite(imgname, frame)
        cv2.imshow('frame', frame) 
        time.sleep(2)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()

In [4]:
LABELIMG_PATH = os.path.join('Tensorflow', 'labelimg')

In [5]:
if not os.path.exists(LABELIMG_PATH):
    !mkdir {LABELIMG_PATH}
    !git clone https://github.com/tzutalin/labelImg {LABELIMG_PATH}

In [6]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

In [7]:
labels = [
    {'name': 'Hello/ Hi', 'id':1},
    {'name': 'How', 'id':2},
    {'name': 'Yes/ Okay', 'id':3}, 
    {'name': 'No', 'id':4}, 
    {'name': 'Nice', 'id':5}, 
    {'name': 'To', 'id':6},
    {'name': 'Meet', 'id':7}, 
    {'name': 'You', 'id':8}, 
    {'name': 'I/ Me', 'id':9}, 
    {'name': 'Thank You', 'id':10}, 
    {'name': 'Peace', 'id':11},
    {'name': 'Love', 'id':12}, 
    {'name': 'Mother', 'id':13}, 
    {'name': 'Father', 'id':14}, 
    {'name': 'Friend', 'id':15}, 
    {'name': 'Please', 'id':16},
    {'name': 'Eat/ Food', 'id':17}, 
    {'name': 'Sorry', 'id':18}, 
    {'name': 'Write', 'id':19}, 
    {'name': 'Help', 'id':20}, 
    {'name': 'Good Job', 'id':21},
    {'name': 'Live Long and Prosper', 'id':22}, 
    {'name': 'Stand', 'id':23}, 
    {'name': 'Up', 'id':24},
]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname: \'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [8]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


In [8]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


In [9]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

In [10]:
!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
#!copy {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

A subdirectory or file Tensorflow\workspace\models\my_ssd_mobnet already exists.


In [11]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [12]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [13]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [14]:
config

{'model': ssd {
   num_classes: 24
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 4e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.01
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.997
         scale: true
         epsilon: 0.001
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
   

In [15]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [16]:
pipeline_config.model.ssd.num_classes = 24
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [17]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

In [24]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=10000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=10000


In [ ]:
#from tensorflow.keras.models import load_model

In [25]:
# def load_pb(path_to_pb):
#     with tf.io.gfile.GFile(path_to_pb, "rb") as f:
#         graph_def = tf.compat.v1.GraphDef()
#         graph_def.ParseFromString(f.read())
#     with tf.Graph().as_default() as graph:
#         tf.import_graph_def(graph_def, name='')
#         return graph

In [21]:
#pwd

'C:\\Users\\parja\\github clones boi\\RealTimeObjectDetection'

In [ ]:
#g = tf.saved_model.load('\\Tensorflow\\workspace\\models\\output_model\\saved_model')

In [ ]:
#model = load_model(f'C:\\Users\\parja\\github clones boi\\RealTimeObjectDetection\\Tensorflow\\workspace\\models\\output_model\\saved_model\\saved_model.pb')

In [ ]:
#!pip3 install tf_slim

In [28]:
#import tf_slim as slim

In [29]:
#from tensorflow.python.ops import image_ops

In [1]:
#import os
from object_detection.utils import label_map_util
from object_detection.builders import model_builder
#from object_detection.utils import visualization_utils as viz_utils

AlreadyExistsError: File system for s3 already registered

In [7]:
import keras
print(keras.__version__)

AlreadyExistsError: Another metric with the same name already exists.

In [30]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-12')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

NameError: name 'model_builder' is not defined

In [32]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [33]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [38]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

NameError: name 'detect_fn' is not defined

In [34]:
detections = detect_fn(input_tensor)

NameError: name 'detect_fn' is not defined